In [1]:
"""# 언어별 분류 작업
- 정확도가 높은 fasttext 모듈로 분류
"""
!pip install fasttext
import copy
import fasttext
import pandas as pd
import re

     |████████████████████████████████| 68 kB 1.8 MB/s eta 0:00:01
  Using cached pybind11-2.8.0-py2.py3-none-any.whl (207 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3290085 sha256=c84ba2abba85d819045a712129f2df1e2c99081675d92e66095452c81a100495
  Stored in directory: /home/lab10/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [18]:
path = '/home/lab10/game_analysis/'
comment_file = f'DokeV_youtube_comment.csv'  
df = pd.read_csv(path+comment_file, encoding='utf-8', header=None)
df.columns=['num','title','comment']
df[:10]

,num,title,comment
0,NaN,title,content
1,0.0,DokeV - World Premiere Gameplay Trailer | game...,Do you remember the amazing longboarding scene...
2,1.0,DokeV - World Premiere Gameplay Trailer | game...,I've never seen a game trailer lag before.\nIt...
3,2.0,DokeV - World Premiere Gameplay Trailer | game...,1:31 어떻게 해서든 저 옷을 입겠다.
4,3.0,DokeV - World Premiere Gameplay Trailer | game...,"This trailer looks awesome, I hope my video wi..."
5,4.0,DokeV - World Premiere Gameplay Trailer | game...,OMG !! My laptop ll Boom \nI m waiting for t...
6,5.0,DokeV - World Premiere Gameplay Trailer | game...,This looks absolutely brilliant.
7,6.0,DokeV - World Premiere Gameplay Trailer | game...,아마 대부분 게이머분들 생각이 저와 비슷할겁니다.\n'한국' 게임업계에 대해 염증까...
8,7.0,DokeV - World Premiere Gameplay Trailer | game...,Not only for classic gamers but for those pare...
9,8.0,DokeV - World Premiere Gameplay Trailer | game...,선생님들 현직 보육교사입니다.. 현재 업계 관계자들이(아기들^^*) 무슨 만화냐고 ...


In [21]:
# 중복 값 제거
print(f"DokeV 전처리 시작")
print('\n')
print('중복 제거 전 :',df.shape)
df = df.drop_duplicates(['comment'],keep='last',ignore_index=True)
print(f'DokeV 중복 제거 후 :',df.shape)
print('\n')

DokeV 전처리 시작


중복 제거 전 : (14543, 3)
DokeV 중복 제거 후 : (14292, 3)




In [22]:
# 소문자로 바꾸기
df['comment'] = df['comment'].str.lower()

In [23]:
# 전처리 전 원본 보존
copy_data = copy.deepcopy(df)

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        "]+", flags=re.UNICODE)

In [24]:
#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

comment_result = []

for i in copy_data['comment'].values:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

In [25]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])

    for p in punct:
        text = text.replace(p, f' {p} ')

    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

cleaned_corpus = []
for sent in comment_result:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [26]:
def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [29]:
basic_preprocessed_corpus = clean_text(cleaned_corpus)
comment_result = pd.DataFrame(basic_preprocessed_corpus, columns=["comment"])

In [30]:
model = fasttext.load_model('/home/lab10/game_analysis/model/lid.176.ftz')

In [31]:
predict = []
for t in comment_result.comment.values:
    predict.append(model.predict(t,k=1))

ty = pd.DataFrame(predict)

In [33]:
comment = []
for num, txt in enumerate(ty[0]):
    txt = str(txt)
    if txt == "('__label__ko',)":
        b = re.sub(txt,"ko",txt)
        comment.append(b)
    elif txt == "('__label__en',)":
        b = re.sub(txt,"en",txt)
        comment.append(b)
    elif txt == "('__label__id',)":
        b = re.sub(txt,"id",txt)
        comment.append(b)
    elif txt == "('__label__es',)":
        b = re.sub(txt,"es",txt)
        comment.append(b)
    else:
        b = re.sub(txt,"etc",txt)
        comment.append(b)

comment = pd.DataFrame(comment)
print('\n')
print(f'DokeV 댓글 언어 구성')
print(comment.value_counts())
print('\n')
pd.set_option('max_columns',50)
pd.set_option('max_rows',100)
# ty_sum.to_csv('ty_sum.csv', encoding='cp949')



DokeV 댓글 언어 구성
(ko)     8843
(en)     3910
(etc)    1024
(id)      344
(es)      171
dtype: int64




In [34]:
title = pd.DataFrame(copy_data['title'])
data = pd.concat([title, comment_result, comment],axis=1)
data.columns = ['title','comment','lang']

In [45]:
path_preprocess='/home/lab10/game_analysis/preprocesing_comment/'
data.to_csv(path_preprocess+'prepro_'+comment_file,  encoding='utf-8', header=None, index=None)
data.drop([0], inplace=True)
print(f"DokeV 전처리 끝")
print('\n')

DokeV 전처리 끝




In [47]:
data.head()

,title,comment,lang
1,DokeV - World Premiere Gameplay Trailer | game...,do you remember the amazing longboarding scene...,(en)
2,DokeV - World Premiere Gameplay Trailer | game...,i ' ve never seen a game trailer lag beforeit ...,(en)
3,DokeV - World Premiere Gameplay Trailer | game...,: 어떻게 해서든 저 옷을 입겠다,(ko)
4,DokeV - World Premiere Gameplay Trailer | game...,this trailer looks awesome i hope my video wil...,(en)
5,DokeV - World Premiere Gameplay Trailer | game...,omg my laptop ll boom i m waiting for this game,(en)


In [11]:
# artist_name=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[공유] Mulcam_Army 공유폴더!/아티스트이름 - Sheet1.csv')
# name=artist_name['engName']
# name.dropna(inplace=True)
# name=name.reset_index(drop=True)
# name.drop([3,5,24,29,37,38,48],axis=0,inplace=True)
# name=name.reset_index(drop=True)
# name[:4]

In [12]:
# for i in range(len(name)):
#     preprocessing(name[i])

In [49]:
# preprocessing('DokeV')